In [13]:
"""
MFP PyTorch Demo - 修复版
严格对齐训练代码的Masking机制
"""

import os
os.chdir("/home/dell/Project-HCL/BaseLine/flexdm_pt/scripts")
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # 强制CPU模式

import json
import logging
from pathlib import Path
from typing import Dict, List, Optional

import torch
import numpy as np
from IPython.display import display, HTML

# ⭐ 导入训练代码中使用的masking模块
from masking_pytorch import (
    get_task_names,
    preprocess_for_train,
    get_seq_mask,
)

print(f"CUDA可用: {torch.cuda.is_available()}")
print(f"设备数量: {torch.cuda.device_count()}")

from models_pytorch import MFP
from dataset import DesignLayoutDataset
from svg_builder_pytorch import SVGBuilder
from retriever_pytorch import ImageRetriever, TextRetriever

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

torch.manual_seed(0)
np.random.seed(0)

print("✓ 导入完成（CPU模式）\n")


# ============================================================
# 配置
# ============================================================
class DemoConfig:
    def __init__(self):
        self.ckpt_dir = "/home/dell/Project-HCL/BaseLine/flexdm_pt/chechpoints"
        self.dataset_name = "crello_json"
        self.db_root = "/home/dell/Project-HCL/BaseLine/flexdm_pt/data/crello_json"
        self.batch_size = 10
        self.device = 'cpu'
        self.target_task = "pos"
        
        self.column_names = {
            "txt": ["gt-layout", "gt-visual", "input", "pred"],
            "img": ["gt-layout", "gt-visual", "input", "pred"],
            "attr": ["gt-layout", "gt-visual", "input", "pred"],
            "pos": ["gt-layout", "gt-visual", "pred-layout", "pred-visual"],
            "elem": ["gt-layout", "gt-visual", "input-layout", "input-visual", 
                     "pred-layout", "pred-visual"],
        }
        
        # ⭐ 任务ID映射（与训练代码一致）
        self.task_name_to_id = {
            "type": 0,
            "pos": 1,
            "attr": 2,
            "img": 3,
            "txt": 4,
            "elem": 5,
        }

config = DemoConfig()
print(f"✓ 配置: device={config.device}, batch_size={config.batch_size}\n")


# ============================================================
# 类型映射
# ============================================================
# DEFAULT_TYPE_MAPPING = {
#     0: '', 1: 'svgElement', 2: 'textElement', 3: 'imageElement',
#     4: 'coloredBackground', 5: 'maskElement', 6: 'humanElement',
# }

DEFAULT_TYPE_MAPPING = {
    0: 'svgElement', 1: 'textElement', 2: 'imageElement', 3: 'coloredBackground',
    4: 'maskElement', 5: ''
}


def load_type_mapping_from_vocab(vocab_file: str):
    return DEFAULT_TYPE_MAPPING

def load_font_mapping_from_dataset(dataset):
    return dataset.idx_to_font if hasattr(dataset, 'idx_to_font') else {}

def load_canvas_mappings_from_dataset(dataset):
    width_mapping = dataset.idx_to_width if hasattr(dataset, 'idx_to_width') else {}
    height_mapping = dataset.idx_to_height if hasattr(dataset, 'idx_to_height') else {}
    return width_mapping, height_mapping


# ============================================================
# 数据处理工具
# ============================================================
def _extract_scalar(tensor):
    if tensor.dim() == 0:
        return tensor.item()
    elif tensor.dim() == 1:
        return tensor[0].item() if tensor.size(0) == 1 else tensor.argmax().item()
    else:
        return tensor.argmax(dim=-1)[0].item() if tensor.size(0) > 0 else 0


def tensor_to_list(data, type_mapping, font_mapping=None, 
                   width_mapping=None, height_mapping=None, bins=64):
    batch_size = data['length'].size(0)
    items = []
    scale = 1.0 / (bins - 1.0)
    
    for i in range(batch_size):
        # Canvas尺寸
        canvas_w = 800
        canvas_h = 600
        
        if 'canvas_width' in data:
            width_idx = data['canvas_width'][i].item()
            if width_mapping and width_idx in width_mapping:
                canvas_w = width_mapping[width_idx]
        
        if 'canvas_height' in data:
            height_idx = data['canvas_height'][i].item()
            if height_mapping and height_idx in height_mapping:
                canvas_h = height_mapping[height_idx]
        
        item = {
            'id': data['id'][i] if 'id' in data else f'sample_{i}',
            'canvas_width': canvas_w,
            'canvas_height': canvas_h,
            'length': data['length'][i].item(),
            'elements': []
        }
        
        for j in range(item['length']):
            element = {}
            
            for key, value in data.items():
                if key in ['id', 'length', 'canvas_width', 'canvas_height']:
                    continue
                if not torch.is_tensor(value):
                    continue
                if value.dim() < 2 or value.size(1) <= j:
                    continue
                
                elem_value = value[i, j]
                
                if key == 'type':
                    element[key] = type_mapping.get(int(_extract_scalar(elem_value)), '')
                elif key == 'font_family' and font_mapping:
                    element[key] = font_mapping.get(int(_extract_scalar(elem_value)), 'Arial')
                elif key in ['left', 'top', 'width', 'height']:
                    discrete_idx = _extract_scalar(elem_value)
                    element[key] = float(np.clip(scale * discrete_idx, 0.0, 1.0))
                elif key == 'color':
                    if elem_value.dim() == 2 and elem_value.size(0) == 3:
                        discrete_indices = elem_value.argmax(dim=-1).cpu().numpy()
                        rgb = [int(idx * 255 / 15) for idx in discrete_indices]
                        element[key] = rgb
                    elif elem_value.dim() == 1 and elem_value.size(0) == 3:
                        rgb = [int(idx * 255 / 15) for idx in elem_value.cpu().numpy()]
                        element[key] = rgb
                    else:
                        element[key] = [128, 128, 128]
                elif key == 'opacity':
                    discrete_idx = _extract_scalar(elem_value)
                    element[key] = float(discrete_idx / 7.0)
                elif 'embedding' in key:
                    element[key] = elem_value.cpu().numpy()
                elif key == 'uuid':
                    element[key] = str(_extract_scalar(elem_value))
                else:
                    element[key] = _extract_scalar(elem_value)
            
            item['elements'].append(element)
        
        items.append(item)
    
    return items


def set_visual_default(item):
    item = item.copy()
    for elem in item.get('elements', []):
        elem.setdefault('color', [128, 128, 128])
        elem.setdefault('opacity', 1.0)
        elem.setdefault('font_family', 'Arial')
    return item


# ============================================================
# 模型相关（纯CPU版本）
# ============================================================
def load_model(checkpoint_path, input_columns):
    """加载模型 - 强制CPU"""
    print(f"加载模型: {checkpoint_path}")
    
    model = MFP(
        input_columns=input_columns,
        embed_dim=256,
        num_blocks=4,
        num_heads=8,
        dropout=0.1,
    )
    
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    # ⭐ 处理不同的checkpoint格式
    if 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
    elif 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = checkpoint
    
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
    
    if missing:
        print(f"  警告: 缺失 {len(missing)} 个键")
    if unexpected:
        print(f"  警告: 多余 {len(unexpected)} 个键")
    
    model.eval()
    print("✓ 模型加载完成（CPU）\n")
    return model


def model_inference_with_masking(model, inputs, input_columns, task_name):
    """
    使用训练代码中相同的masking机制进行推理
    
    Args:
        model: MFP模型
        inputs: 输入数据字典
        input_columns: 输入列配置
        task_name: 任务名称 ('pos', 'elem', 'attr', 'txt', 'img')
    
    Returns:
        outputs: 模型预测结果
        modified_inputs: 应用mask后的输入
        masks: mask字典
    """
    # ⭐ 获取任务ID（与训练代码一致）
    task_name_to_id = {
        "type": 0, "pos": 1, "attr": 2, 
        "img": 3, "txt": 4, "elem": 5,
    }
    task_id = task_name_to_id.get(task_name, 1)
    
    # ⭐ 使用训练代码中的preprocess_for_train函数
    targets, modified_inputs, masks = preprocess_for_train(
        inputs,
        input_columns,
        task_id,
        is_autoreg=False,  # demo使用非自回归模式
    )
    
    # 确保所有数据在CPU上
    for key in modified_inputs:
        if torch.is_tensor(modified_inputs[key]):
            modified_inputs[key] = modified_inputs[key].cpu()
    
    # 模型推理
    with torch.no_grad():
        outputs = model(modified_inputs)
    
    return outputs, modified_inputs, masks


# ============================================================
# 可视化主函数（修复版）
# ============================================================
def visualize_reconstruction(model, example, builders, config, input_columns,
                            type_mapping, font_mapping=None, width_mapping=None, 
                            height_mapping=None, bins=64):
    """
    可视化重建结果 - 使用与训练代码一致的masking机制
    """
    svgs = []
    target_task = config.target_task
    
    # 确保数据在CPU
    for key in example:
        if torch.is_tensor(example[key]):
            example[key] = example[key].cpu()
    
    items = tensor_to_list(example, type_mapping, font_mapping, 
                          width_mapping, height_mapping, bins)
    
    print(f"渲染 {len(items)} 个样本...")
    
    # GT
    print("  - GT Layout")
    svgs.append([builders["layout"](item) for item in items])
    print("  - GT Visual")
    svgs.append([builders["visual"](item) for item in items])
    
    # 输入视图（根据任务类型显示）
    if target_task == "txt":
        print("  - Input (无文本)")
        svgs.append([builders["visual_wo_text"](item) for item in items])
    elif target_task == "img":
        print("  - Input (无图像)")
        svgs.append([builders["visual_wo_image"](item) for item in items])
    elif target_task == "attr":
        print("  - Input (默认属性)")
        svgs.append([builders["visual"](set_visual_default(item)) for item in items])
    
    # ⭐ 使用训练代码的masking机制进行推理
    print(f"  - 模型推理 (任务: {target_task})")
    outputs, modified_inputs, masks = model_inference_with_masking(
        model, example, input_columns, target_task
    )
    
    # 元素级任务的输入视图
    if target_task == "elem":
        # 显示去掉第一个元素后的输入
        example_copy = {}
        for key, value in modified_inputs.items():
            if isinstance(value, torch.Tensor) and value.dim() >= 2:
                if value.size(1) > 1:
                    example_copy[key] = value[:, 1:]  # 去掉第一个元素
                else:
                    example_copy[key] = value
            else:
                example_copy[key] = value
        
        # 调整length
        if 'length' in example:
            example_copy['length'] = torch.clamp(example['length'] - 1, min=1)
        
        items_copy = tensor_to_list(example_copy, type_mapping, font_mapping,
                                    width_mapping, height_mapping, bins)
        print("  - Input Layout (去掉第一个元素)")
        svgs.append([builders["layout"](item) for item in items_copy])
        print("  - Input Visual (去掉第一个元素)")
        svgs.append([builders["visual"](item) for item in items_copy])
    
    # ⭐ 合并预测结果和原始输入
    pred = {}
    for key in example:
        if key in outputs:
            pred[key] = outputs[key]
        else:
            pred[key] = example[key]
    
    # 对于categorical类型，需要从logits转换
    for key, value in outputs.items():
        column = input_columns.get(key, {})
        if column.get('type') == 'categorical' and value.dim() >= 3:
            # 如果是logits，取argmax
            if value.size(-1) > 1:
                pred[key] = value.argmax(dim=-1)
    
    pred_items = tensor_to_list(pred, type_mapping, font_mapping,
                                width_mapping, height_mapping, bins)
    
    # 预测结果视图
    if target_task in ["pos", "elem"]:
        print("  - Pred Layout")
        svgs.append([builders["layout"](item) for item in pred_items])
    
    print("  - Pred Visual")
    svgs.append([builders["visual"](item) for item in pred_items])
    
    print("✓ 渲染完成\n")
    return list(zip(*svgs))


# ============================================================
# 主流程
# ============================================================
print("="*80)
print("MFP PyTorch Demo - 修复版（对齐训练代码）")
print("="*80)

# 1. 加载数据集
print("\n1. 加载数据集")
dataset = DesignLayoutDataset(config.db_root, split='test', max_length=20)
print(f"✓ 数据集大小: {len(dataset)}\n")

# 2. 加载映射
print("2. 加载映射")
vocab_file = Path(config.db_root).parent / "vocabulary.json"
type_mapping = load_type_mapping_from_vocab(str(vocab_file))
font_mapping = load_font_mapping_from_dataset(dataset)
width_mapping, height_mapping = load_canvas_mappings_from_dataset(dataset)

if font_mapping:
    print(f"✓ 字体映射: {len(font_mapping)} 个字体")
if width_mapping:
    print(f"✓ Width映射: {len(width_mapping)} 个尺寸")
if height_mapping:
    print(f"✓ Height映射: {len(height_mapping)} 个尺寸")
print()

# 3. 创建DataLoader
print("3. 创建DataLoader")
from torch.utils.data import DataLoader
from dataset import collate_fn

dataloader = DataLoader(dataset, batch_size=config.batch_size, 
                       shuffle=False, collate_fn=collate_fn)
example = next(iter(dataloader))

# 强制转CPU
for key in example:
    if torch.is_tensor(example[key]):
        example[key] = example[key].cpu()

print(f"✓ 批次形状: {example['left'].shape}\n")

# 4. 加载配置
print("4. 加载模型配置")
input_columns_file = './input_columns_dataset.json'

with open(input_columns_file, 'r') as f:
    input_columns = json.load(f)

# ⭐ 打印任务信息（与训练代码一致）
task_names = get_task_names(input_columns)
print(f"✓ 输入列数: {len(input_columns)}")
print(f"✓ 任务列表: {task_names}\n")

# 5. 加载模型
print("5. 加载模型")
checkpoint_path = "/home/dell/Project-HCL/BaseLine/flexdm_pt/checkpoints/retrain_v1/best.pth"
model = load_model(str(checkpoint_path), input_columns)

# 6. 构建检索数据库
print("6. 构建检索数据库")
db_root = Path(config.db_root).parent / config.dataset_name
image_db = ImageRetriever(db_root, image_path=db_root / "images")
image_db.build("test")
text_db = TextRetriever(db_root, text_path=db_root / "texts")
text_db.build("test")
print()

# 7. 创建SVG构建器
print("7. 创建SVG构建器")
builders = {
    "layout": SVGBuilder(key='type', max_width=128, max_height=192, opacity=0.8),
    "visual": SVGBuilder(key='color', max_width=128, max_height=192, 
                        image_db=image_db, text_db=text_db, 
                        render_text=True, opacity=1.0),
    "visual_wo_text": SVGBuilder(key='color', max_width=128, max_height=192,
                                 image_db=image_db, text_db=None, 
                                 render_text=False, opacity=1.0),
    "visual_wo_image": SVGBuilder(key='color', max_width=128, max_height=192,
                                  image_db=None, text_db=text_db, 
                                  render_text=True, opacity=1.0),
}
print("✓ 构建器创建完成\n")

# 8. 运行可视化
print("="*80)
print(f"8. 开始可视化 - 任务: {config.target_task}")
print("="*80)

svgs = visualize_reconstruction(model, example, builders, config, input_columns,
                                type_mapping, font_mapping, width_mapping, 
                                height_mapping, bins=64)

print(f"✓ 生成了 {len(svgs)} 个样本的可视化结果")
print(f"列名: {config.column_names[config.target_task]}\n")

# 9. 显示结果
print("="*80)
print("可视化结果:")
print("="*80)

for i, row in enumerate(svgs[:10]):
    print(f"\nSample {i}:")
    display(HTML("<div style='margin: 10px 0;'>%s</div>" % " ".join(row)))

print("\n✓ Demo完成!")

CUDA可用: False
设备数量: 0
✓ 导入完成（CPU模式）

✓ 配置: device=cpu, batch_size=10

MFP PyTorch Demo - 修复版（对齐训练代码）

1. 加载数据集
加载数据: /home/dell/Project-HCL/BaseLine/flexdm_pt/data/crello_json/test.json
✓ 加载了 2248 个样本

构建查找表...
  Type词汇表: 5 个类型
  Type映射: {'svgElement': 0, 'textElement': 1, 'imageElement': 2, 'coloredBackground': 3, 'maskElement': 4, '<UNKNOWN>': 0}
  Canvas Width词汇表: 41 个尺寸
  Canvas Height词汇表: 46 个尺寸
  Font词汇表: 34 个字体 (OOV->0)
✓ 数据集大小: 2248

2. 加载映射
✓ 字体映射: 34 个字体
✓ Width映射: 42 个尺寸
✓ Height映射: 47 个尺寸

3. 创建DataLoader
✓ 批次形状: torch.Size([10, 20, 1])

4. 加载模型配置
✓ 输入列数: 15
✓ 任务列表: ['random', 'elem', 'type', 'pos', 'attr', 'img', 'txt']

5. 加载模型
加载模型: /home/dell/Project-HCL/BaseLine/flexdm_pt/checkpoints/retrain_v1/best.pth

初始化MFP模型
初始化Encoder:
  type: Embedding(8, 256)
  left: Embedding(66, 256)
  top: Embedding(66, 256)
  width: Embedding(66, 256)
  height: Embedding(66, 256)
  opacity: Embedding(10, 256)
  color: Embedding(18, 256)
  image_embedding: Linear(512, 256)
  text_embedding: 

RuntimeError: Error(s) in loading state_dict for MFP:
	size mismatch for encoder.emb_layers.0.weight: copying a param with shape torch.Size([7, 256]) from checkpoint, the shape in current model is torch.Size([8, 256]).
	size mismatch for decoder.head_layers.0.weight: copying a param with shape torch.Size([5, 256]) from checkpoint, the shape in current model is torch.Size([6, 256]).
	size mismatch for decoder.head_layers.0.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([6]).